In [8]:
import pandas as pd;
import os ;
import json;
import logging
from sqlalchemy import create_engine
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import  numpy as np
from tensorflow import keras
import tensorflow_text
import string
import random
import uuid


In [19]:
class Multi_Sentiment_Classifier:
    def __init__(self,module):
        self.model= keras.models.load_model('Multi-Language_Sentiment_Classifier/sentiment_classifier2')
        print(self.model)
        self.module=module

    def sentences_vectors(self,sentences, batch: int = 1000):
        """
        Compute the sentence embedding in batches to make it memory
        efficient.
        :param sentences: iterable of strings
        :param batch: batches size to compute the embeddings.The smaller
        the longer it takes.
        :return:  numpy nmatrix  of shape (X,512)
        """

        lower = 0
        upper = batch
        sent_vectors = self.module.signatures['question_encoder'](
            tf.constant(sentences[lower:upper]))["outputs"]
        while upper < len(sentences):
            lower += batch
            upper += batch
            print(lower)
            if sentences:
                sent_vectors = np.concatenate(
                    (sent_vectors, self.module.signatures['question_encoder'](
                        tf.constant(sentences[lower:upper]))["outputs"]))
        return sent_vectors

    def sentiment(self,s_setence: str) -> str:
        s_vector = self.sentences_vectors([s_setence])
        s_prediction = (self.model.predict(s_vector.numpy()) > 0.5).astype("int32")
        s_prediction = np.argmax(s_prediction, axis=1)
        s_prob = self.model.predict(s_vector.numpy())
        
        decode = {1: 'positive', 0: 'negative'}
        sentiment = decode[s_prediction[0]] if s_prob.max() >= 0.7 else 'neutral'
        return sentiment



In [10]:
module = hub.load('encoder')

In [20]:
sentiment_classifier=Multi_Sentiment_Classifier(module);

In [23]:
sentiment_classifier.sentiment("i love u")

'positive'

In [24]:
def sentiment_analysese_list(comment_df):
    comment_df['sentiment']=comment_df.apply(lambda x: sentiment_classifier.sentiment(x['text']), axis=1)
    return comment_df;
    

In [25]:
import shortuuid

alphabet = string.ascii_lowercase + string.digits
su = shortuuid.ShortUUID(alphabet=alphabet)

def random_choice():
    return ''.join(random.choices(alphabet, k=8))


In [26]:
import mysql.connector
def import_hotel_info_to_DB(hotel_info,hotel_id,cursor):
    print(hotel_info)
    name=hotel_info['name'].replace("'", " ");
    location=hotel_info['location'].replace("'", " ");
    staff='';
    facilities=''
    cleanliness=''
    comfort=''
    valueformoney=''
    location=''
    freeWiFi=''
    
    if "Staff" in hotel_info['sous_Category']:
        staff=hotel_info['sous_Category']['Staff']
    else:
        staff="0"
    if "Facilities" in hotel_info['sous_Category']:
        facilities=hotel_info['sous_Category']['Facilities']
    else:
        facilities="0"
    if "Cleanliness" in hotel_info['sous_Category']:
        cleanliness=hotel_info['sous_Category']['Cleanliness']
    else:
        cleanliness="0"
    if "Comfort" in hotel_info['sous_Category']:
        comfort=hotel_info['sous_Category']['Comfort']
    else:
        comfort="0"
    if "Valueformoney" in hotel_info['sous_Category']:
        valueformoney=hotel_info['sous_Category']['Valueformoney']
    else:
        valueformoney="0"
    if "Location" in hotel_info['sous_Category']:
        location=hotel_info['sous_Category']['Location']
    else:
        location="0"
    if "FreeWiFi" in hotel_info['sous_Category']:
        freeWiFi=hotel_info['sous_Category']['FreeWiFi']
    else:
        freeWiFi="0"
    #----------------------dim-hotel----------------------
    query="INSERT INTO \
    `dim_hotel`( \
        `Id_hotel`,\
        `Hotel_name`, \
        `Adress`,  \
        `Score`, \
        `Staff`,  \
        `Facilities`,\
        `Cleanliness`, \
        `Comfort`,  \
        `Value_for_money`, \
        `Location`,   \
        `Free_WiFi`   \
        ) \
        VALUES   \
        (   '{}','{}', '{}', '{}','{}','{}','{}','{}','{}','{}','{}')".format(hotel_id,name,location,hotel_info['Score'],
                                                                              staff,facilities,cleanliness
                                                                             ,comfort,valueformoney,location,freeWiFi);
    cursor.execute(query)    

In [27]:
import mysql.connector


def import_comment_to_DB(comment_json,hotel_id,cursor):
    id_tmp=random_choice();
    j=1;
    for ind in comment_json.index:
        #--------------------dim_date
        id_time=str(j)+hotel_id[1]+hotel_id[0]+id_tmp+"T"
        tmp=comment_json['visite_date'][ind].split(' ')
        date_query="INSERT INTO\
                    `dim_date`(`Id_time`, `year`, `month`, `day`)\
                    VALUES\
                    ('{}','{}','{}','{}')".format(id_time,tmp[1],tmp[0],1);
        cursor.execute(date_query)
        #--------------------dim_room
        id_room=str(j)+hotel_id[1]+hotel_id[0]+id_tmp+"Ro"
        room_query="INSERT INTO\
        `dim_room`(`Id_room`, `Room_type`)\
        VALUES\
        ('{}', '{}')".format(id_room,comment_json['room_type'][ind]);
        cursor.execute(room_query)
        #--------------------dim_dim_reviewer
        id_rv=str(j)+hotel_id[1]+hotel_id[0]+id_tmp+"RV"
        nbr_nights=comment_json['number_of_nights'][ind].split(" ")
        name=comment_json['name'][ind].replace("'",' ');
        name=name.replace("/",'');
        name=name.replace("\\",'');
        rv_query="INSERT INTO\
        `dim_reviewer`(\
        `Id_reviewer`,\
        `Reviewer_name`,\
        `Reviewer_nationality`,\
        `Number_of_night`,\
        `Type_of_stay`,\
        `Visite_date`\
        )\
        VALUES\
        ('{}','{}','{}','{}','{}','{}')".format(id_rv,name,comment_json['nationality'][ind], nbr_nights[0],comment_json['clinet_type'][ind],comment_json['visite_date'][ind]);
        cursor.execute(rv_query)
        #--------------------table_de_fait
        text=comment_json['text'][ind].replace("'", "")
        fact_query="INSERT INTO  `fact_table`(`Id_hotel`,`Id_reviewer`, `Id_room`,`Id_time`,`Review_score`,`Comment`,`emotions`)VALUES(\
        '{}','{}','{}','{}','{}','{}','{}')".format(hotel_id,id_rv,id_room,id_time,comment_json['score'][ind],text,comment_json['sentiment'][ind]);
        cursor.execute(fact_query)
        j+=1;


In [28]:
def load(hotel_name,path_list,hotel_id):
    print("---------->"+path_list[0])
    print("----------->"+path_list[1])
    info_file=open(path_list[0]);
    info_file=json.load(info_file)
    comment_file=open(path_list[1]);
    comment_file=json.load(comment_file);
    comment_df=pd.DataFrame.from_records(comment_file)
    comment_df=comment_df.replace('There are no comments available for this review','001')
    comment_df=sentiment_analysese_list(comment_df);
    connection = mysql.connector.connect(host='localhost',
                                         database='test_database',
                                         user='root',
                                         password='')
    cursor = connection.cursor()        
    import_hotel_info_to_DB(info_file,hotel_id,cursor)
    import_comment_to_DB(comment_df,hotel_id,cursor)
    connection.commit()
    connection.close()
    print(comment_df)
    return comment_df
    
    

In [29]:
path='D:\hotel_old'
list_dir=os.listdir(path)


In [30]:
i:int=1;

for dir in list_dir:
        try:
            randomLetter = random.choice(string.ascii_letters)
            id_tmp=random_choice()
            hotel_id="{}".format(i)+dir[0]+randomLetter+ id_tmp+"H"
    
            name_list=os.listdir(path+"\\"+dir)
            file_list=[]
            for el in name_list:
                file_list.append(path+"\\"+dir+"\\"+el)
            load(dir,file_list,hotel_id)
            i+=1;
        except:
            print("--------------------->Error {}".format(dir));
   



---------->D:\hotel_old\Appart Hotel Monaco\1_Appart Hotel Monaco_info.json
----------->D:\hotel_old\Appart Hotel Monaco\2_Appart Hotel Monaco_comment.json
{'name': 'Condo Hotel\nAppart Hotel Monaco', 'location': 'Boulevard arabie saudi L.karima 3 LT.84, 90000 Tangier, Morocco', 'Score': '8.2', 'sous_Category': {'Staff': '8.6', 'Facilities': '8.0', 'Cleanliness': '8.3', 'Comfort': '8.2', 'Valueformoney': '8.1', 'Location': '8.1', 'FreeWiFi': '10'}}
         name               nationality           number_of_nights  \
0      Fatima                    Sweden       1 night ·  July 2021   
1        Isou                   Morocco    2 nights ·  August 2021   
2      Hassan  United States of America      3 nights ·  July 2021   
3       Yahia                    Canada       7 nights ·  May 2021   
4       Akmal                  Portugal  1 night ·  September 2019   
..        ...                       ...                        ...   
73         Si                   Germany     1 night ·  Au

{'name': 'Hotel Atlas Amadil Beach Hotel', 'location': 'Chemin Des Dunes Agadir - Maroc, 80000 Agadir, Morocco', 'Score': '8.3', 'sous_Category': {'Staff': '8.6', 'Facilities': '8.1', 'Cleanliness': '8.4', 'Comfort': '8.4', 'Valueformoney': '8.0', 'Location': '9.3', 'FreeWiFi': '8.4'}}
                 name     nationality           number_of_nights clinet_type  \
0              Meryem          France      3 nights ·  July 2020      Family   
1                  Rm         Ireland  3 nights ·  November 2021      Family   
2              Eferri         Germany  5 nights ·  November 2021       Group   
3               Allan  United Kingdom  1 night ·  September 2021       Group   
4             Laveena  United Kingdom   3 nights ·  October 2021       Group   
...               ...             ...                        ...         ...   
1899    Travelzone.pl         twitter                        nan         nan   
1900            IZIVA         twitter                        nan         

{'name': 'Hotel Aymane ROOFTOP Hotel', 'location': 'Bab souk, 91000 Chefchaouen, Morocco', 'Score': '9.8', 'sous_Category': {'Staff': '9.8', 'Facilities': '9.6', 'Cleanliness': '9.8', 'Comfort': '9.8', 'Valueformoney': '9.9', 'Location': '9.8', 'FreeWiFi': '10'}}
         name               nationality            number_of_nights  \
0       Sanae                   Germany  2 nights ·  September 2021   
1     Mohamed                   Morocco    1 night ·  December 2021   
2    Delphine                   Belgium    1 night ·  November 2021   
3   Guillaume                    Canada   2 nights ·  November 2021   
4      Robert  United States of America    1 night ·  November 2021   
..        ...                       ...                         ...   
95  Anonymous                   Morocco       1 night ·  April 2021   
96  Anonymous                   Morocco       1 night ·  March 2021   
97  Anonymous                   Morocco    1 night ·  February 2021   
98  Anonymous             

{'name': 'Hotel Be Live Collection Saïdia', 'location': 'Parcela H7 - Station Balnéaire, 60600 Saïdia, Morocco', 'Score': '7.1', 'sous_Category': {'Staff': '7.6', 'Facilities': '7.0', 'Cleanliness': '7.2', 'Comfort': '7.3', 'Valueformoney': '6.3', 'Location': '8.2', 'FreeWiFi': '6.2'}}
          name               nationality           number_of_nights  \
0        Tarik                    Russia    3 nights ·  August 2021   
1       Aichab                   Morocco    7 nights ·  August 2021   
2       Khalid                   Morocco      5 nights ·  June 2019   
3       Yassir  United States of America      4 nights ·  July 2021   
4           Mk  United States of America      3 nights ·  June 2021   
..         ...                       ...                        ...   
254  Anonymous                   Morocco  1 night ·  September 2021   
255  Anonymous                   Morocco  1 night ·  September 2021   
256  Anonymous                   Morocco  1 night ·  September 2021   
257

{'name': 'Hotel Casa Las Tortugas Petit Beach Hotel & Spa', 'location': 'Calle Igualdad sin numero, Manzana 1 Lote 2, 77310 Holbox Island, Mexico', 'Score': '9.4', 'sous_Category': {'Staff': '9.4', 'Facilities': '9.4', 'Cleanliness': '9.6', 'Comfort': '9.6', 'Valueformoney': '8.6', 'Location': '9.8', 'FreeWiFi': '4.4'}}
          name               nationality           number_of_nights  \
0        Phong                   Germany   8 nights ·  October 2021   
1        Erica  United States of America   4 nights ·  January 2022   
2        Irene               Netherlands   3 nights ·  January 2022   
3     Jonathan                   Belgium  3 nights ·  December 2021   
4     Jonathan            United Kingdom   1 night ·  December 2021   
..         ...                       ...                        ...   
185     Fabien                    France  3 nights ·  February 2019   
186     Sandro               Switzerland  2 nights ·  December 2021   
187    Lourdes                     Spai

...

{'name': 'Hotel Radisson Blu Hotel Casablanca City Center', 'location': '02 Blvd Mohamed Diouri, Sidi Belyout, 20250 Casablanca, Morocco', 'Score': '8.4', 'sous_Category': {'Staff': '9.1', 'Facilities': '8.6', 'Cleanliness': '9.0', 'Comfort': '9.0', 'Valueformoney': '8.2', 'Location': '7.8', 'FreeWiFi': '8.7'}}
                     name               nationality  \
0                  Faisal                     Qatar   
1                    Davy                   Belgium   
2                 Charles                   Morocco   
3                    Wole                   Nigeria   
4                 Jocelyn  United States of America   
...                   ...                       ...   
1007            Lian El's                   twitter   
1008         Tara Kitchen                   twitter   
1009  Hotel News Resource                   twitter   
1010              ttgmena                   twitter   
1011           Luxe Radio                   twitter   

               number_of_n

{'name': 'Hotel Relax Airport', 'location': 'Aeroport International Mohamed V Nouasseur B-P 101, 20 000 Nouaseur, Morocco', 'Score': '8.0', 'sous_Category': {'Staff': '8.4', 'Facilities': '8.0', 'Cleanliness': '8.3', 'Comfort': '8.2', 'Valueformoney': '7.7', 'Location': '8.7', 'FreeWiFi': '8.0'}}
                   name           nationality           number_of_nights  \
0             Abdelaziz                 Spain  1 night ·  September 2021   
1               Sara808           Philippines   1 night ·  November 2021   
2                 Tariq          Saudi Arabia   1 night ·  December 2021   
3                 Ahmed  United Arab Emirates   1 night ·  November 2021   
4               Alathea               Morocco   1 night ·  November 2021   
...                 ...                   ...                        ...   
2341         FunCoupons               twitter                        nan   
2342           Blogosum               twitter                        nan   
2343  Spotlight Be

{'name': 'Condo Hotel\nRésidence La Place', 'location': 'Place Oum Essaad,Avenue Afghanistan, 70000 Laayoune, Morocco', 'Score': '8.2', 'sous_Category': {'Staff': '8.9', 'Facilities': '8.4', 'Cleanliness': '8.5', 'Comfort': '8.6', 'Valueformoney': '8.0', 'Location': '8.7', 'FreeWiFi': '7.9'}}
         name     nationality          number_of_nights    clinet_type  \
0     Mohamed      Mauritania    5 nights ·  April 2019         Couple   
1        Rene         Denmark   2 nights ·  August 2021  Solo traveler   
2      Ismail  United Kingdom   1 night ·  October 2020  Solo traveler   
3       Steve  United Kingdom  2 nights ·  January 2020          Group   
4     Tatiana          Russia  1 night ·  December 2019         Family   
..        ...             ...                       ...            ...   
92     Cheikh         Morocco      1 night ·  July 2019  Solo traveler   
93     Fattah         Morocco     3 nights ·  June 2019         Couple   
94     Oumama         Morocco       1 ni

{'name': 'Hotel STARS HOTEL', 'location': 'Angle rue sebou et Rue Ouadi Naffis, Guéliz, 40000 Marrakesh, Morocco', 'Score': '8.1', 'sous_Category': {'Staff': '8.9', 'Facilities': '8.4', 'Cleanliness': '8.8', 'Comfort': '8.7', 'Valueformoney': '8.7', 'Location': '9.1', 'FreeWiFi': '7.7'}}
                            name nationality           number_of_nights  \
0                         Khalil     Morocco    1 night ·  January 2022   
1                           Saïd     Morocco   5 nights ·  January 2022   
2                           Omar     Morocco   1 night ·  December 2021   
3                         Meriem     Morocco   1 night ·  December 2021   
4                         Yassir     Morocco  4 nights ·  December 2021   
...                          ...         ...                        ...   
4304   Matt Fayne ''STUPID SOB''     twitter                        nan   
4305           Jennifer Eberhard     twitter                        nan   
4306                 Drew Grgich    

{'name': 'Hotel Suites Hotel Mohammed V by Accor', 'location': 'Place De La Marche Verte, 32 000 Al Hoceïma, Morocco', 'Score': '8.4', 'sous_Category': {'Staff': '8.8', 'Facilities': '8.1', 'Cleanliness': '8.4', 'Comfort': '8.7', 'Valueformoney': '7.9', 'Location': '9.3', 'FreeWiFi': '8.4'}}
          name     nationality           number_of_nights    clinet_type  \
0        Layla  United Kingdom      2 nights ·  July 2019         Family   
1       Annika          Sweden    1 night ·  October 2021          Group   
2    Stephanie         Morocco     3 nights ·  April 2021         Couple   
3        Najib         Morocco     3 nights ·  March 2021  Solo traveler   
4        Jacek         Morocco    1 night ·  January 2020          Group   
..         ...             ...                        ...            ...   
292  Anonymous     Netherlands   1 night ·  February 2019         Couple   
293  Anonymous         Morocco    1 night ·  October 2019         Family   
294  Anonymous         